In [2]:
import numpy as np
import scipy
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import os
from utils import get_sample_freq, get_test_freq, get_stimulus, remove_outliers
from scipy.stats import ttest_ind, levene

mode = 'Frequency'
starting_point_types = [0.0,0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0]
window_size_types = [1.0,1.5,2.0,2.5,3.0]

stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

In [3]:
for starting_point, window_size in product(starting_point_types,window_size_types):
    #mode = 'Frequency'
    #print(f'Start point: {starting_point}s after a stimulus')
    #print(f'Window Size: {window_size}s')
    #print(f'End point: {starting_point + window_size}s after a stimulus')
    os.makedirs(f'./sample_{mode}/data_sp{starting_point}ws{window_size}', exist_ok=True)
    os.makedirs(f'./test_{mode}/data_sp{starting_point}ws{window_size}', exist_ok=True)

    for i in range(1, 11):
        df_sample = get_sample_freq(i, start_gap=starting_point, window_size= window_size, mode = mode)
        df_test = get_test_freq(i, start_gap=starting_point, window_size= window_size, mode = mode)

        df_sample.to_csv(f'./sample_{mode}/data_sp{starting_point}ws{window_size}/sample_spike_neuron{i}.csv')
        df_test.to_csv(f'./test_{mode}/data_sp{starting_point}ws{window_size}/test_spike_case{i}.csv', index = False)




In [4]:
result_DataFrame = pd.DataFrame(columns= ['Starting Point', 'Window Size'] + [f'Case {i+1}' for i in range(10)])
for starting_point, window_size in product(starting_point_types,window_size_types):
    row = {'Starting Point': [starting_point], 'Window Size': [window_size]}
    significance_level = 0.1
    ans = []
    for test_num in range(1, 11):
        test_spike = pd.read_csv(f'./test_{mode}/data_sp{starting_point}ws{window_size}/test_spike_case{test_num}.csv')
        candidate = []
        for i in range(1, 11):
            sample_spike = pd.read_csv(f'./sample_{mode}/data_sp{starting_point}ws{window_size}/sample_spike_neuron{i}.csv')
            p_list = []
            for degree in np.unique(stimulus[:,1]):
                test = remove_outliers(test_spike[test_spike['Degree'] == int(degree)][mode].values)
                sample = remove_outliers(sample_spike[sample_spike['Degree'] == int(degree)][mode].values)
                _, p_eqv = levene(test, sample)
                #if p_eqv < 0.05:
                _, p = ttest_ind(test, sample)
                p_list.append(p >= significance_level)
            if sum(p_list) > 6:
                candidate.append(i)
        if len(candidate) == 0:
            ans.append(None)
        elif len(candidate) == 1:
            ans.append(candidate[0])
        elif len(candidate) > 1:
            ans.append(tuple(candidate))
    for i, val in enumerate(ans):
        row[f'Case {i+1}'] = [val]
    
    if None not in ans:
        result_DataFrame = pd.concat([result_DataFrame, pd.DataFrame(row)], axis=0)
result_DataFrame.to_csv('evenmoreAblation_noNone_noOutliers.csv', index = False)

    #if None not in ans:
    #    print(f'Starting_point: {starting_point}, Window_size: {window_size}...')
    #    print(ans)

    
    #if all(p_list) & (len(ans) == 9) & (len(ans) == len(set(ans))):
    #    print(f'Starting_point: {starting_point}, Window_size: {window_size}')
    #    print(ans)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom
/Library/Frameworks/Python.framework/Versions

In [28]:
from scipy.spatial.distance import jensenshannon
from utils import remove_outliers

def to_density(spike):
    pdf = []
    for degree in np.unique(stimulus[:,1]):
        frequency = (spike[spike['Degree'] == int(degree)]['Counts'] / 2).values
        frequency = remove_outliers(frequency)
        pdf.append(np.mean(frequency)) #+ 1e-6)
    
    pdf = np.asarray(pdf) 
    return pdf 
Case = [1, 6, 7]
Neuron = [1, 2, 6]

for c, n in product(np.arange(1,11), np.arange(1,11)):

    sample = pd.read_csv(f'./sample_Frequency/data_sp0.1ws2.0/sample_spike_neuron{c}.csv')
    test = pd.read_csv(f'./test_Frequency/data_sp0.1ws2.0/test_spike_case{n}.csv')

    sample_mass = to_density(sample)
    test_mass = to_density(test)
    assert len(sample_mass) == len(test_mass)
    dist = jensenshannon(sample_mass, test_mass)
    #print(f'Case {c} vs Neuron {n}', jensenshannon(sample_mass, test_mass))

    



Case 1 vs Neuron 1 0.16582208457450348
Case 1 vs Neuron 2 0.40710267028405855
Case 1 vs Neuron 6 0.10425414377359325
Case 6 vs Neuron 1 0.12791257468439357
Case 6 vs Neuron 2 0.4685046103129579
Case 6 vs Neuron 6 0.15156499478176264
Case 7 vs Neuron 1 0.3311854839052789
Case 7 vs Neuron 2 0.26677049113228685
Case 7 vs Neuron 6 0.27825949685233187


In [29]:
rule = 'Counts/data_sp0.1ws2.0'
#pd.read_csv(f'./sample_Counts/data_sp0.1ws2.0/sample_spike_neuron{c}.csv')
#test = pd.read_csv(f'./test_Counts/data_sp0.1ws2.0/test_spike_case{n}.csv')

os.makedirs(f'./sample_{rule}/images/')
for sample_num in range(1,11):
    plt.figure(figsize=(5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./sample_{rule}/sample_spike_neuron{sample_num}.csv'), x = 'Degree', y = 'Counts').set(title= f'Neuron {sample_num}')
    plt.tight_layout()
    plt.savefig(f'./sample_{rule}/images/sample_spike_neuron{sample_num}.png')
    plt.close()

In [46]:
arr = np.array([9, 0, 4,  7, 3,  6])
q = 20
print(arr[arr > np.percentile(arr, q)])

[9 4 7 6]


In [39]:
for n in range(1,11):
    print('Neuron ', n)
    data = pd.read_csv(f'./sample_{rule}/sample_spike_neuron{n}.csv')
    for degree in np.unique(stimulus[:,1]):
        print(degree)
        print((data[data['Degree'] == degree]['Counts'].values))

Neuron  1
0.0
[9 0 4 7 3 6]
20.0
[ 2 10 10  4  8  6  4 10]
40.0
[18  9 11 11  6 22  8]
60.0
[24 24 15 26 10]
80.0
[46 23 38  5 48  6 19 53]
100.0
[18  9 37 42  6 36  6]
120.0
[26  0 12  6 14  8]
140.0
[17 10  5  1 10  5  5]
160.0
[4 3 3 4 0 6]
Neuron  2
0.0
[2 3 1 0 2 1]
20.0
[1 0 4 5 2 1 2 1]
40.0
[ 0  3  1 19  3  8  8]
60.0
[ 3 10  5 10  8]
80.0
[39  6 48  0 39  1 14 72]
100.0
[ 1  5 84 91  3 90  8]
120.0
[29  2 35 12 52  9]
140.0
[ 6  6  4  1 10  7  5]
160.0
[1 1 1 1 0 4]
Neuron  3
0.0
[103  20  90 112  13  95]
20.0
[15 35 45  7 25 42 58 10]
40.0
[11 12 15  7  7  6 22]
60.0
[ 2  1  4  4 45]
80.0
[ 5  8 16  7  1  9  9 12]
100.0
[10 13  7  4 10 18  5]
120.0
[12 12  2 26 17 20]
140.0
[20 39 20 26 11 21 17]
160.0
[59 12 12 53 13 50]
Neuron  4
0.0
[8 0 0 0 3 1]
20.0
[0 0 0 0 0 0 0 2]
40.0
[1 1 0 1 0 1 3]
60.0
[3 0 1 0 2]
80.0
[1 0 0 0 0 0 0 0]
100.0
[0 1 1 0 2 0 0]
120.0
[ 1  6  1 11  6 19]
140.0
[12 16 43 24  6 22 33]
160.0
[12 11 47 12 19 12]
Neuron  5
0.0
[11 19 19 20 18 17]
20.0
[14 

/Users/vuno/Documents/Projects/CNeuro_winterschool/CompNeuro2024/utils.py:63: RuntimeWarning: invalid value encountered in divide
  z_scores = np.abs((arr - np.mean(arr)) / np.std(arr))


In [37]:
stimulus

array([[  2.632478,  60.      ],
       [  7.635518, 140.      ],
       [ 12.638598, 120.      ],
       [ 17.641638,   0.      ],
       [ 22.644718, 120.      ],
       [ 27.647758,  60.      ],
       [ 32.650838, 160.      ],
       [ 37.653918, 100.      ],
       [ 42.656958, 160.      ],
       [ 47.660038,  40.      ],
       [ 52.663078, 140.      ],
       [ 57.666158,   0.      ],
       [ 62.669198, 100.      ],
       [ 67.672278,  20.      ],
       [ 72.675358,  80.      ],
       [ 77.678398,  20.      ],
       [ 82.681478,  40.      ],
       [ 87.684518,  80.      ],
       [ 92.687598, 140.      ],
       [ 97.690678,  60.      ],
       [102.693718,  20.      ],
       [107.696798,  40.      ],
       [112.699878,  40.      ],
       [117.702918, 100.      ],
       [122.705998,  80.      ],
       [127.709078, 140.      ],
       [132.712118,   0.      ],
       [137.715198, 100.      ],
       [142.718278, 160.      ],
       [147.721318,  60.      ],
       [15

In [32]:
os.makedirs(f'./test_{rule}/images/', exist_ok=True)
for num in range(1, 11):
    plt.figure(figsize = (5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./test_{rule}/test_spike_case{num}.csv'), x = 'Degree', y = 'Counts', color='orange').set(title = f'Case {num}')
    plt.tight_layout()
    plt.savefig(f'./test_{rule}/images/test_spike_case{num}.png')
    plt.close()

In [20]:
sample_spike ##= pd.read_csv()
print(remove_outliers(test_spike[test_spike['Degree'] == int(160)][mode].values))
#test = remove_outliers(test_spike[test_spike['Degree'] == int(degree)][mode].values)
#sample = remove_outliers(sample_spike[sample_spike['Degree'] == int(degree)][mode].values)

[204 133 192 128  91 147]


In [25]:
new = list(np.arange(1,11))
new.remove(8)
print(new.remove(8))

ValueError: list.remove(x): x not in list

In [17]:
stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

In [18]:
stimulus

array([[  2.632478,  60.      ],
       [  7.635518, 140.      ],
       [ 12.638598, 120.      ],
       [ 17.641638,   0.      ],
       [ 22.644718, 120.      ],
       [ 27.647758,  60.      ],
       [ 32.650838, 160.      ],
       [ 37.653918, 100.      ],
       [ 42.656958, 160.      ],
       [ 47.660038,  40.      ],
       [ 52.663078, 140.      ],
       [ 57.666158,   0.      ],
       [ 62.669198, 100.      ],
       [ 67.672278,  20.      ],
       [ 72.675358,  80.      ],
       [ 77.678398,  20.      ],
       [ 82.681478,  40.      ],
       [ 87.684518,  80.      ],
       [ 92.687598, 140.      ],
       [ 97.690678,  60.      ],
       [102.693718,  20.      ],
       [107.696798,  40.      ],
       [112.699878,  40.      ],
       [117.702918, 100.      ],
       [122.705998,  80.      ],
       [127.709078, 140.      ],
       [132.712118,   0.      ],
       [137.715198, 100.      ],
       [142.718278, 160.      ],
       [147.721318,  60.      ],
       [15

In [14]:
np.arange(0,0.55,0.05)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ])

In [15]:
np.arange(1,3.5,0.5)

array([1. , 1.5, 2. , 2.5, 3. ])

In [2]:
for i in range(1, 11):
    df = get_sample_freq(i)
    df.to_csv(f'./sample/sample_spike_neuron{i}.csv', index = False)

In [3]:
for i in range(1, 11):
    df = get_test_freq(i)
    df.to_csv(f'./test/test_spike_case{i}.csv', index = False)

In [32]:
for sample_num in range(1,11):
    plt.figure(figsize=(5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./sample/sample_spike_neuron{sample_num}.csv'), x = 'Degree', y = 'Frequency').set(title= f'Neuron {sample_num}')
    plt.tight_layout()
    plt.savefig(f'./sample/images/sample_spike_neuron{sample_num}.png')
    plt.close()

In [35]:
for num in range(1, 11):
    plt.figure(figsize = (5,5), dpi = 150)
    sns.barplot(data = pd.read_csv(f'./test/test_spike_case{num}.csv'), x = 'Degree', y = 'Frequency', color='orange').set(title = f'Case {num}')
    plt.tight_layout()
    plt.savefig(f'./test/images/test_spike_case{num}.png')
    plt.close()

In [2]:
stimulus_raw = scipy.io.loadmat('dat_stimulus.mat')
spk_sample_raw = scipy.io.loadmat('dat_spk_sample.mat')
spk_test_raw = scipy.io.loadmat('dat_spk_test.mat')

In [3]:
stimulus = stimulus_raw['stimulus']
spk_sample = spk_sample_raw['spk_sample']
spk_test = spk_test_raw['spk_test']

In [5]:
stimulus

array([[  2.632478,  60.      ],
       [  7.635518, 140.      ],
       [ 12.638598, 120.      ],
       [ 17.641638,   0.      ],
       [ 22.644718, 120.      ],
       [ 27.647758,  60.      ],
       [ 32.650838, 160.      ],
       [ 37.653918, 100.      ],
       [ 42.656958, 160.      ],
       [ 47.660038,  40.      ],
       [ 52.663078, 140.      ],
       [ 57.666158,   0.      ],
       [ 62.669198, 100.      ],
       [ 67.672278,  20.      ],
       [ 72.675358,  80.      ],
       [ 77.678398,  20.      ],
       [ 82.681478,  40.      ],
       [ 87.684518,  80.      ],
       [ 92.687598, 140.      ],
       [ 97.690678,  60.      ],
       [102.693718,  20.      ],
       [107.696798,  40.      ],
       [112.699878,  40.      ],
       [117.702918, 100.      ],
       [122.705998,  80.      ],
       [127.709078, 140.      ],
       [132.712118,   0.      ],
       [137.715198, 100.      ],
       [142.718278, 160.      ],
       [147.721318,  60.      ],
       [15

In [7]:
degree_dict = dict()
for degree in np.unique(stimulus[:,1]):
    stim_time_list = stimulus[stimulus[:,1] == degree,0]
    degree_dict[degree] = stim_time_list


In [8]:
degree_dict

{0.0: array([ 17.641638,  57.666158, 132.712118, 172.736678, 202.755078,
        232.773518, 302.816518, 357.850318, 387.868758, 407.881078,
        492.733358, 527.754878, 542.764118, 607.804078, 647.828718,
        707.865598]),
 20.0: array([ 67.672278,  77.678398, 102.693718, 177.739758, 227.770438,
        247.782718, 287.807318, 307.819598, 392.871838, 432.896438,
        467.717958, 512.745638, 582.788718, 612.807158, 637.822558,
        677.847158]),
 40.0: array([ 47.660038,  82.681478, 107.696798, 112.699878, 237.776598,
        262.791958, 317.825758, 327.831878, 362.853398, 412.884158,
        457.711798, 522.751798, 552.770238, 627.816398, 697.859478,
        717.871758]),
 60.0: array([  2.632478,  27.647758,  97.690678, 147.721318, 197.752038,
        267.794998, 297.813438, 352.847238, 367.856478, 372.859558,
        472.721038, 537.761038, 587.791798, 592.794878, 662.837918,
        692.856398]),
 80.0: array([ 72.675358,  87.684518, 122.705998, 152.724398, 252.785798,

In [20]:
def get_sample_freq(sample_num, start_gap=0.1, window_size = 2):
    df_deg_freq = pd.DataFrame(columns=['Time','Degree','Frequency'])
    sample = spk_sample[sample_num-1,0]

    i = 0
    for time, degree in stimulus:
        if (time >= 50) & (time <= 350):
            frequency = len(sample[(sample > (time+start_gap)) & (sample <= (time + start_gap + window_size))]) / window_size
            df_deg_freq.loc[i,:] = [time, int(degree), frequency]
            i += 1
    return df_deg_freq


            



In [72]:
from scipy.stats import ttest_ind

significance_level = 0.1
for test_num in range(1, 11):
    test_spike = pd.read_csv(f'./test/test_spike_case{test_num}.csv')
    for i in range(1, 11):
        sample_spike = pd.read_csv(f'./sample/sample_spike_neuron{i}.csv')
        p_list = []
        for degree in np.unique(stimulus[:,1]):
            test = test_spike[test_spike['Degree'] == int(degree)]['Frequency']
            sample = sample_spike[sample_spike['Degree'] == int(degree)]['Frequency']
            _, p = ttest_ind(test, sample)
            p_list.append(p >= significance_level)
        #print(sum(p_list))
        if sum(p_list) > 7:
        #if all(p_list):
            print(f'Case {test_num} matches with Neuron {i}')



Case 2 matches with Neuron 3
Case 3 matches with Neuron 7
Case 4 matches with Neuron 8
Case 5 matches with Neuron 5
Case 6 matches with Neuron 6
Case 7 matches with Neuron 2


In [75]:
from scipy.stats import entropy

def to_density(spike):
    pdf = []
    for degree in np.unique(stimulus[:,1]):
        pdf.append(spike[spike['Degree'] == int(degree)]['Frequency'].mean() + 1e-6)
    
    pdf = np.asarray(pdf) 
    pdf /= (np.sum(pdf) + len(pdf)*1e-6)
    return pdf

for test_num in range(1, 11):
    test_spike = pd.read_csv(f'./test/test_spike_case{test_num}.csv')
    test_pdf = to_density(test_spike)

    KLdiv_list = []
    for i in range(1, 11):
        sample_spike = pd.read_csv(f'./sample/sample_spike_neuron{i}.csv')
        sample_pdf = to_density(sample_spike)
        KLdiv = entropy(pk=sample_pdf, qk = test_pdf)
        KLdiv_list.append(KLdiv)
    print(f'Case {test_num} matches with Neuron {np.argmin(KLdiv_list) + 1}') 

Case 1 matches with Neuron 6
Case 2 matches with Neuron 3
Case 3 matches with Neuron 7
Case 4 matches with Neuron 2
Case 5 matches with Neuron 5
Case 6 matches with Neuron 1
Case 7 matches with Neuron 2
Case 8 matches with Neuron 10
Case 9 matches with Neuron 9
Case 10 matches with Neuron 10


In [43]:
array = [True, True, False]
print(all(array))

False


In [21]:
for i in range
get_sample_freq(1)

In [22]:
df

,Time,Degree,Frequency
0,52.663078,140,8.5
1,57.666158,0,4.5
2,62.669198,100,9.0
3,67.672278,20,1.0
4,72.675358,80,23.0
5,77.678398,20,5.0
6,82.681478,40,9.0
7,87.684518,80,11.5
8,92.687598,140,5.0
9,97.690678,60,12.0


In [1]:

sample_num = 9


start_gap = 0.1
window_size = 2
end_time = start_gap + window_size
sample = spk_sample[sample_num-1,0]

#index = 
for key in degree_dict:
    print(key)
    stim_list = degree_dict[key]
    assert len(stim_list) == 16
    frequency_list = []
    for stim_time in stim_list:
        if (stim_time >= 50) & (stim_time <= 350):
            frequency = len(sample[(sample > (stim_time+start_gap)) & (sample <= (stim_time + end_time))]) / window_size
            frequency_list.append(frequency)
        #else:
        #    continue
    print(frequency_list)
    print(np.round(np.mean(frequency_list), 3), np.round(np.std(frequency_list), 3))

#df_deg_freq.to_csv(f'./sample/sample_spike_neuron{sample_num}.csv', index = False)

NameError: name 'spk_sample' is not defined

In [73]:
sample

array([[ 51.04463],
       [ 51.04744],
       [ 51.87194],
       ...,
       [349.45813],
       [349.56484],
       [349.61507]])

In [1]:
import pandas as pd

In [4]:
import numpy as np 

X =np.array([[1,2],[3,4]])
Y = np.array([[3,4],[1,2]])

#X = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
#Y = np.array([[5,6,7,8],[9,10,11,12],[1,2,3,4]]) #+ np.random.randn(3,4)*0.01

In [5]:
np.linalg.pinv(Y)

array([[ 1. , -2. ],
       [-0.5,  1.5]])

In [34]:
P = np.array([[0,0,1],[1,0,0],[0,1,0]])
print(P @ Y)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [6]:
P_psd = np.matmul(X ,np.linalg.pinv(Y)) #@ Y

In [7]:
P_psd

array([[ 2.22044605e-16,  1.00000000e+00],
       [ 1.00000000e+00, -8.88178420e-16]])

In [37]:
[1/3, -1/6, 5/6]

[0.3333333333333333, -0.16666666666666666, 0.8333333333333334]

In [39]:
import numpy as np

def rref(matrix):
    lead = 0
    rowCount, colCount = matrix.shape

    for r in range(rowCount):
        if lead >= colCount:
            return matrix

        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rowCount:
                i = r
                lead += 1
                if colCount == lead:
                    return matrix

        matrix[[r, i]] = matrix[[i, r]]
        if matrix[r, lead] != 0:
            matrix[r] = matrix[r] / matrix[r, lead]

        for i in range(rowCount):
            if i != r:
                matrix[i] = matrix[i] - matrix[i, lead] * matrix[r]

        lead += 1

    return matrix

# Example usage:
A = np.array([[2, 1, -1, 8],
              [-3, -1, 2, -11],
              [-2, 1, 2, -3]])

rref_A = rref(A)

print("Original matrix:")
print(A)
print("\nReduced Row Echelon Form:")
print(rref_A)

Original matrix:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]

Reduced Row Echelon Form:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]


In [47]:
import numpy as np

def rref_without_permutation(matrix):
    lead = 0
    rowCount, colCount = matrix.shape

    for r in range(rowCount):
        if lead >= colCount:
            return matrix

        # Find the first non-zero entry in the current column
        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rowCount:
                i = r
                lead += 1
                if colCount == lead:
                    return matrix

        # Normalize the pivot row
        matrix[r] = matrix[r] / matrix[r, lead]

        # Eliminate non-zero entries above and below the pivot
        for i in range(rowCount):
            if i != r:
                matrix[i] = matrix[i] - matrix[i, lead] * matrix[r]

        lead += 1

    return matrix

# Example usage:
A = np.array([[2, 1, -1, 8],
              [-3, -1, 2, -11],
              [-2, 1, 2, -3]])

rref_A = rref_without_permutation(A)

print("Original matrix:")
print(A)
print("\nReduced Row Echelon Form without permutation:")
print(rref_A)

Original matrix:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]

Reduced Row Echelon Form without permutation:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]


In [51]:
A = np.array([[2, 1, -1, 8],
              [-3, -1, 2, -11],
              [-2, 1, 2, -3]])
print(A)
rref_A = rref_without_permutation(A)

print("Original matrix:")
print(A)
print("\nReduced Row Echelon Form without permutation:")
print(rref_A)

[[  2   1  -1   8]
 [ -3  -1   2 -11]
 [ -2   1   2  -3]]
Original matrix:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]

Reduced Row Echelon Form without permutation:
[[1 0 0 4]
 [0 1 0 1]
 [0 0 1 1]]


In [53]:
P_psd

array([[ 0.33333333, -0.16666667,  0.83333333],
       [ 0.33333333,  0.33333333,  0.33333333],
       [ 0.33333333,  0.83333333, -0.16666667]])

In [54]:
print(rref_without_permutation(P_psd))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0. -0.  1.]]


In [40]:
rref(P_psd)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [-0., -0.,  1.]])

In [26]:
_X = np.array([[1,2,3],[5,6,7],[9,10,11]])
_Y = np.array([[5,6,7],[9,10,11],[1,2,3]]) 
print(np.matmul(_X, np.linalg.inv(_Y)))

[[ 0.   -0.25  1.  ]
 [ 0.   -0.5   0.75]
 [ 0.    0.    0.75]]


In [24]:
np.linalg.det(_X)

-7.105427357600985e-15

In [11]:
(_X @ np.linalg.inv(_Y)) @ _Y

array([[-1.25, -0.5 ,  0.25],
       [-3.75, -3.5 , -3.25],
       [ 0.75,  1.5 ,  2.25]])